In [13]:
from IPython.display import display, Markdown
def md(s):
    display(Markdown(s))

import numpy as np

API_URL = "http://localhost:11434/api/generate"

SYSTEM_PROMPT = """You are a helpful AI assistant that can call functions when appropriate. If the question asked does not relate to the functions, do not respond in a JSON format. Respond in JSON format with the following structure:

If a function call is needed:
{
  "action": "call_function",
  "function": "<function_name>",
  "arguments": {"arg1": value1, "arg2": value2, ...}
}

Available functions:
- toothnumbers: Calculate gear diameters. Arguments: m (module in mm, float), N1 (teeth on driving gear, int), gearratio (target gear ratio, float).
- Add: Add two numbers. Arguments: x (float), y (float).
- Square: Square a number. Arguments: x (float).

"""

def getinput(prompt):
    try:
        return input(prompt)
    except EOFError:
        return None
    except KeyboardInterrupt:  # When ESC is pressed
        return None

In [14]:
def display_conversation_history(messages):
    """
    Displays conversation history in a two-column markdown table in a Jupyter notebook.
    
    Args:
        messages: List of strings alternating between user queries and LLM responses
    """
    from IPython.display import clear_output
    # Clear the output to avoid clutter
    clear_output(wait=True)
    # Create table header
    table = "| User Query | LLM Response |\n"
    table += "|------------|-------------|\n"
    
    # Add rows to the table, processing messages in pairs
    for i in range(0, len(messages), 2):
        user_msg = messages[i]
        
        # Check if there's a corresponding LLM response
        llm_msg = messages[i+1] if i+1 < len(messages) else ""
        
        # Add row to table, escaping any pipe characters in the messages
        user_msg_escaped = user_msg.replace("|", "\\|")
        llm_msg_escaped = llm_msg.replace("|", "\\|")
        
        table += f"| {user_msg_escaped} | {llm_msg_escaped} |\n"
    
    # Display the table using the md() function
    md(table)

In [15]:

def toothnumbers(m, N1, gearratio):
    """
    Calculate the diameters of the gears in a gear train.

    Parameters:
    m (float): The module of the gears (mm)
    N1 (int): The number of teeth on the driving gear.
    gearratio (float): The target gear ratio.

    Returns:
    dict: A dictionary containing gear details and the actual gear ratio.
    """
    d1 = m * N1  # Diameter of the driving gear
    N2 = int(N1 * gearratio)  # Number of teeth on the driven gear
    d2 = m * N2  # Diameter of the driven gear
    actual_gear_ratio = N2 / N1  # Actual gear ratio

    if abs(actual_gear_ratio - gearratio) > 0.01:
        raise ValueError(f"Gear ratio mismatch: {actual_gear_ratio:.2f} != {gearratio:.2f}")

    return {
        "text": "Computation successful",
        "number_of_teeth_gear1": N1,
        "diameter_gear1_mm": d1,
        "number_of_teeth_gear2": N2,
        "diameter_gear2_mm": d2,
        "actual_gear_ratio": actual_gear_ratio
    }

def add(x, y):
    addition = x+y
    return {
        "Addition result": addition
    }

def square(x):
    square = x**2
    return {
        "Square result": square
    }


functions = {
    "toothnumbers": toothnumbers,
    "Add": add,
    "Square": square
    # Add more functions here as needed, e.g., "another_function": another_function
}

In [16]:
import requests

def generate_response(prompt):
    full_prompt = f"{SYSTEM_PROMPT}\n\nUser: {prompt}\nAssistant:"
    
    try:
        response = requests.post(
            API_URL,
            json={
                "model": "llama3.2:3b",
                "prompt": full_prompt,
                "stream": False
            },
            timeout=60  # Increased timeout to 60 seconds
        )
        response.raise_for_status()
        result = response.json()
        return result.get("response", "No response")
    except requests.Timeout:
        return "Error: Request timed out. The model is taking too long to respond. Please try again."
    except requests.ConnectionError:
        return "Error: Could not connect to the Ollama server. Please ensure it is running."
    except requests.RequestException as e:
        return f"Error connecting to Llama model: {str(e)}"

In [17]:
import re
import ast
import json

def extract_json(response_text):
    """ Checks if the response by the model is in JSON format, and returns the response"""
    start = response_text.find('{')
    end = response_text.rfind('}') + 1
    if start != -1 and end != -1:
        json_str = response_text[start:end]
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return None
    return None

CHAT_HISTORY = []

while True:
    user_input = getinput("Enter a prompt (or 'q' to quit): ")
    if user_input in (None, "q", "Q"):
        break

    CHAT_HISTORY.append(user_input)
    response = generate_response(user_input)

    json_response = extract_json(response) # checks if the llm responds with a json as expected
    if json_response:
        action = json_response.get("action")
        if action == "call_function":
            function_name = json_response.get("function")
            arguments = json_response.get("arguments", {})
            if function_name in functions:
                try:
                    result = functions[function_name](**arguments)
                    if function_name == "toothnumbers":
                        final_response = (
                            f"{result['text']}: Driver Gear: {result['number_of_teeth_gear1']} teeth, "
                            f"{result['diameter_gear1_mm']:.1f}mm diameter, Driven Gear: "
                            f"{result['number_of_teeth_gear2']} teeth, {result['diameter_gear2_mm']:.1f}mm diameter, "
                            f"Actual Gear Ratio: {result['actual_gear_ratio']:.3f}"
                        )
                    elif function_name == "add":
                        final_response = f"{result['add']}"
                    elif function_name == "square":
                        final_response = f"{result['square']}"
                    else:
                        final_response = str(result)  # Default formatting for other functions
                except Exception as e:
                    final_response = f"Error in function call: {str(e)}"
            else:
                final_response = f"Unknown function: {function_name}"
        elif action == "respond":
            final_response = json_response.get("text", "")
        else:
            final_response = "Invalid action in response."
    else:
        final_response = f"Could not parse JSON from response. Raw response: {response}"

    CHAT_HISTORY.append(final_response)
    display_conversation_history(CHAT_HISTORY)
    

| User Query | LLM Response |
|------------|-------------|
| add 2 and 3 | {'Addition result': 5} |
| square 3456 | {'Square result': 11943936} |
